In [2]:
# Import necessary libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Step 1: Load and preprocess datasets
def load_datasets():
    try:
        # Define file paths for the datasets
        crops_file = 'final_karnataka_dataset.csv'
        nutrients_file = 'nutrients.csv'
        age_nutrients_file = 'age_wise_nutrients_consumtion_final.csv'
        
        # Load datasets into Pandas DataFrames
        crops_df = pd.read_csv(crops_file)
        nutrients_df = pd.read_csv(nutrients_file)
        age_nutrients_df = pd.read_csv(age_nutrients_file)
        
        # Standardize column names: remove units in parentheses and convert to lowercase
        crops_df.columns = crops_df.columns.str.lower()
        nutrients_df.columns = nutrients_df.columns.str.lower()
        age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()
        
        # Convert all string values in the dataframes to lowercase
        crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        
        print("Datasets loaded and preprocessed successfully!")
        return crops_df, nutrients_df, age_nutrients_df
    except Exception as e:
        print(f"Error loading datasets: {e}")
        return None, None, None

# Load datasets
crops_df, nutrients_df, age_nutrients_df = load_datasets()

Datasets loaded and preprocessed successfully!


/tmp/ipykernel_417273/3940261154.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  crops_df = crops_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_417273/3940261154.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nutrients_df = nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
/tmp/ipykernel_417273/3940261154.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  age_nutrients_df = age_nutrients_df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

def nutrient_analysis_with_exchange(crops_df, nutrients_df, age_nutrients_df):
    # Step 1: Dynamic User Input for District Name
    district_name = input("Enter the district name: ").strip().lower()  # Normalize input for consistency

    # Validate district existence
    if district_name not in crops_df['district_name'].str.lower().unique():
        print(f"District '{district_name}' not found in the dataset. Please check the input.")
        return None

    print(f"\nDistrict '{district_name}' found in the dataset. Proceeding with analysis...\n")

    # Filter crop data for the selected district
    selected_district_crops = crops_df[crops_df['district_name'].str.lower() == district_name]

    # Merge crop data with nutrient dataset to calculate total nutrients
    district_nutrient_data = selected_district_crops.merge(nutrients_df, on='crop', how='inner')
    nutrient_columns = ['carbohydrates', 'protein', 'fiber', 'fat', 'vitamin a', 'vitamin c',
                        'iron', 'calcium', 'potassium', 'magnesium']
    nutrient_totals = district_nutrient_data[nutrient_columns].sum()
    print(f"Total nutrients available in district '{district_name}':")
    print(nutrient_totals)

    # Step 2: Calculate Nutrient Deficiencies
    deficiencies = {}
    age_nutrients_df.columns = age_nutrients_df.columns.str.replace(r"\s*\(.*\)", "", regex=True).str.strip().str.lower()
    for age_group in age_nutrients_df['group']:
        age_rdi = age_nutrients_df[age_nutrients_df['group'] == age_group][nutrient_columns]
        deficiency = age_rdi.values[0] - nutrient_totals.values
        deficiency_dict = {col: val for col, val in zip(nutrient_columns, deficiency) if val > 0}
        deficiencies[age_group] = deficiency_dict

    # Identify most deficient group
    high_deficiency_groups = {group: sum(def_dict.values()) for group, def_dict in deficiencies.items() if def_dict}
    if not high_deficiency_groups:
        print("No significant deficiencies found.")
        return None

    most_deficient_group = max(high_deficiency_groups, key=high_deficiency_groups.get)
    print(f"\nMost Deficient Group: {most_deficient_group}")
    print("Deficient Nutrients:")
    for nutrient, amount in deficiencies[most_deficient_group].items():
        print(f"- {nutrient}: {amount:.2f}")

    # Step 3: Recommend Crops for Deficient Nutrients
    print("\nSuggested Crops for Deficient Nutrients:")
    recommended_crops = {}
    for nutrient, deficit in deficiencies[most_deficient_group].items():
        nutrient_rich_crops = nutrients_df[nutrients_df[nutrient] > 0].sort_values(by=nutrient, ascending=False)
        top_crops = nutrient_rich_crops[['crop', nutrient]].head(3).to_dict(orient='records')  # Top 3 crops
        recommended_crops[nutrient] = top_crops
        print(f"- {nutrient}:")
        for crop in top_crops:
            print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")

    # Step 4: Project Future Deficiencies (10-year Projection)
    growth_rate = 0.05  # Assuming a default annual growth rate of 5%
    projected_deficiencies = {}
    print("\n10-Year Projected Deficiency:")
    for nutrient, current_deficiency in deficiencies[most_deficient_group].items():
        future_deficiency = current_deficiency * (1 + growth_rate) ** 10
        projected_deficiencies[nutrient] = future_deficiency
        print(f"- {nutrient}: {future_deficiency:.2f}")

    # Step 5: Analyze Neighboring Districts for Surplus Crops
    print("\nNeighboring Districts with Surplus Crops:")
    surplus_crops = {}
    neighboring_districts = crops_df[crops_df['state_name'] == selected_district_crops['state_name'].iloc[0]]['district_name'].unique().tolist()
    neighboring_districts.remove(district_name)
    for nutrient in deficiencies[most_deficient_group]:
        surplus_crops[nutrient] = []
        for neighbor in neighboring_districts:
            neighbor_crops = crops_df[crops_df['district_name'].str.lower() == neighbor.lower()]
            neighbor_data = neighbor_crops.merge(nutrients_df, on='crop', how='inner')
            if neighbor_data[nutrient].sum() > deficiencies[most_deficient_group][nutrient]:
                surplus_crops[nutrient].append({
                    'district': neighbor,
                    'crops': neighbor_data[['crop', nutrient]].to_dict(orient='records'),
                    'surplus': neighbor_data[nutrient].sum()
                })
    for nutrient, districts in surplus_crops.items():
        print(f"- {nutrient}:")
        for district in districts:
            print(f"  District: {district['district']}, Surplus: {district['surplus']}, Crops: {district['crops']}")

    # Step 6: Suggest Crop Exchange Policy
    print("\nSuggested Crop Exchanges:")
    exchange_suggestions = {}
    for nutrient, districts in surplus_crops.items():
        if districts:
            donor = districts[0]  # Take the first district with surplus as the donor
            exchange_suggestions[nutrient] = {
                'receiving_district': district_name,
                'donor_district': donor['district'],
                'crops_to_exchange': donor['crops'],
                'surplus_utilized': deficiencies[most_deficient_group][nutrient],
                'remaining_surplus': donor['surplus'] - deficiencies[most_deficient_group][nutrient]
            }
    for nutrient, suggestion in exchange_suggestions.items():
        print(f"- {nutrient}: Donor District: {suggestion['donor_district']}, Remaining Surplus: {suggestion['remaining_surplus']}")
        print(f"  Crops to Exchange: {suggestion['crops_to_exchange']}")

    return {
        "most_deficient_group": most_deficient_group,
        "deficiencies": deficiencies[most_deficient_group],
        "recommended_crops": recommended_crops,
        "projected_deficiencies": projected_deficiencies,
        "surplus_crops": surplus_crops,
        "exchange_suggestions": exchange_suggestions
    }

# Example Usage
results = nutrient_analysis_with_exchange(crops_df, nutrients_df, age_nutrients_df)

# Display Results
if results:
    print("\n=== Output Summary ===")
    print(f"Most Deficient Group: {results['most_deficient_group']}")
    print("\nDeficient Nutrients:")
    for nutrient, amount in results['deficiencies'].items():
        print(f"- {nutrient}: {amount:.2f}")
    print("\nSuggested Crops:")
    for nutrient, crops in results['recommended_crops'].items():
        print(f"- {nutrient}:")
        for crop in crops:
            print(f"  * {crop['crop']} (Nutrient Value: {crop[nutrient]})")
    print("\n10-Year Projected Deficiencies:")
    for nutrient, value in results['projected_deficiencies'].items():
        print(f"- {nutrient}: {value:.2f}")
    print("\nSurplus Crops from Neighboring Districts:")
    for nutrient, districts in results['surplus_crops'].items():
        print(f"- {nutrient}:")
        for district in districts:
            print(f"  District: {district['district']}, Crops: {district['crops']}, Surplus: {district['surplus']}")
    print("\nExchange Suggestions:")
    for nutrient, suggestion in results['exchange_suggestions'].items():
        print(f"- {nutrient}: From {suggestion['donor_district']} to {suggestion['receiving_district']}")
        print(f"  Crops to Exchange: {suggestion['crops_to_exchange']}, Remaining Surplus: {suggestion['remaining_surplus']}")


Enter the district name: chitradurga

District 'chitradurga' found in the dataset. Proceeding with analysis...

Total nutrients available in district 'chitradurga':
carbohydrates     29695.0
protein            8174.3
fiber              4033.9
fat                9922.6
vitamin a        101234.0
vitamin c          4081.2
iron               2527.4
calcium             126.2
potassium        312725.0
magnesium         86609.0
dtype: float64

Most Deficient Group: young male crossbred (1000 number)
Deficient Nutrients:
- carbohydrates: 109470305.00
- protein: 21031825.70
- fiber: 12495966.10
- fat: 25490077.40
- vitamin a: 328398766.00
- vitamin c: 29245918.80
- iron: 4397472.60
- calcium: 474499873.80
- potassium: 1715187275.00
- magnesium: 148163391.00

Suggested Crops for Deficient Nutrients:
- carbohydrates:
  * other fibres (Nutrient Value: 80.0)
  * ginger (Nutrient Value: 80.0)
  * jute (Nutrient Value: 76.0)
- protein:
  * soyabean (Nutrient Value: 36.0)
  * rapeseed & mustard (Nutri